# Pivot & Fibonacci vs High & Low - Analytics

### Import Library

In [34]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [35]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [36]:
data = dataset.copy()

In [37]:
data["ema_f2"] = ta.ema(data["Close"], length=16)
data["ema_l2"] = ta.ema(data["Close"], length=250)
data["long_trend"] = data.apply(lambda r: 'up' if r['ema_f2'] > r['ema_l2'] else 'down', axis=1)
data.drop(columns=['ema_f2', 'ema_l2'], inplace=True)
data.dropna(inplace=True)

In [38]:
def cal_fibonacci(htd):
    _1_d_df = htd.resample("D").agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'long_trend': 'first'
    })
    _1_d_df.dropna(inplace=True)
    _1_d_df["high_low"] = _1_d_df.apply(lambda r: r['High'] - r['Low'], axis=1)
    _1_d_df["P"] = _1_d_df.apply(lambda r: (r['High'] + r['Low'] + r['Close']) / 3, axis=1)
    _1_d_df["R1"] = _1_d_df.apply(lambda r: r['P'] + 0.236 * r['high_low'], axis=1)
    _1_d_df["R2"] = _1_d_df.apply(lambda r: r['P'] + 0.382 * r['high_low'], axis=1)
    _1_d_df["R3"] = _1_d_df.apply(lambda r: r['P'] + 0.5 * r['high_low'], axis=1)
    _1_d_df["R4"] = _1_d_df.apply(lambda r: r['P'] + 0.618 * r['high_low'], axis=1)
    _1_d_df["R5"] = _1_d_df.apply(lambda r: r['P'] + 0.786 * r['high_low'], axis=1)
    _1_d_df["R6"] = _1_d_df.apply(lambda r: r['P'] + r['high_low'], axis=1)

    _1_d_df["S1"] = _1_d_df.apply(lambda r: r['P'] - 0.236 * r['high_low'], axis=1)
    _1_d_df["S2"] = _1_d_df.apply(lambda r: r['P'] - 0.382 * r['high_low'], axis=1)
    _1_d_df["S3"] = _1_d_df.apply(lambda r: r['P'] - 0.5 * r['high_low'], axis=1)
    _1_d_df["S4"] = _1_d_df.apply(lambda r: r['P'] - 0.618 * r['high_low'], axis=1)
    _1_d_df["S5"] = _1_d_df.apply(lambda r: r['P'] - 0.786 * r['high_low'], axis=1)
    _1_d_df["S6"] = _1_d_df.apply(lambda r: r['P'] - r['high_low'], axis=1)
    
    _1_d_df.dropna(inplace=True)
    return _1_d_df

In [39]:
_1_d_df = cal_fibonacci(data)
_1_d_df

,Open,Close,High,Low,long_trend,high_low,P,R1,R2,R3,R4,R5,R6,S1,S2,S3,S4,S5,S6
Date,,,,,,,,,,,,,,,,,,,
2018-08-13,943.5,954.2,954.6,942.3,down,12.3,950.366667,953.269467,955.065267,956.516667,957.968067,960.034467,962.666667,947.463867,945.668067,944.216667,942.765267,940.698867,938.066667
2018-08-14,955.5,959.3,961.8,952.1,down,9.7,957.733333,960.022533,961.438733,962.583333,963.727933,965.357533,967.433333,955.444133,954.027933,952.883333,951.738733,950.109133,948.033333
2018-08-15,958.0,946.6,962.3,946.6,down,15.7,951.833333,955.538533,957.830733,959.683333,961.535933,964.173533,967.533333,948.128133,945.835933,943.983333,942.130733,939.493133,936.133333
2018-08-16,942.4,947.0,947.3,935.5,down,11.8,943.266667,946.051467,947.774267,949.166667,950.559067,952.541467,955.066667,940.481867,938.759067,937.366667,935.974267,933.991867,931.466667
2018-08-17,952.2,947.0,954.6,945.0,down,9.6,948.866667,951.132267,952.533867,953.666667,954.799467,956.412267,958.466667,946.601067,945.199467,944.066667,942.933867,941.321067,939.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-29,1321.1,1326.8,1330.1,1320.6,up,9.5,1325.833333,1328.075333,1329.462333,1330.583333,1331.704333,1333.300333,1335.333333,1323.591333,1322.204333,1321.083333,1319.962333,1318.366333,1316.333333
2024-08-30,1328.4,1332.7,1334.9,1328.0,up,6.9,1331.866667,1333.495067,1334.502467,1335.316667,1336.130867,1337.290067,1338.766667,1330.238267,1329.230867,1328.416667,1327.602467,1326.443267,1324.966667
2024-09-04,1320.8,1314.0,1321.5,1307.3,up,14.2,1314.266667,1317.617867,1319.691067,1321.366667,1323.042267,1325.427867,1328.466667,1310.915467,1308.842267,1307.166667,1305.491067,1303.105467,1300.066667


In [40]:
def get_zone(r, column):
    zone = 0
    if r[column] < r['S6']:
        zone = -7
    elif r['S6'] <= r[column] < r['S5'] :
        zone = -6
    elif r['S5'] <= r[column] < r['S4'] :
        zone = -5
    elif r['S4'] <= r[column] < r['S3'] :
        zone = -4
    elif r['S3'] <= r[column] < r['S2'] :
        zone = -3
    elif r['S2'] <= r[column] < r['S1'] :
        zone = -2
    elif r['S1'] <= r[column] < r['P'] :
        zone = -1
    elif r['P'] <= r[column] < r['R1'] :
        zone = 1
    elif r['R1'] <= r[column] < r['R2'] :
        zone = 2
    elif r['R2'] <= r[column] < r['R3'] :
        zone = 3
    elif r['R3'] <= r[column] < r['R4'] :
        zone = 4
    elif r['R4'] <= r[column] < r['R5'] :
        zone = 5
    elif r['R5'] <= r[column] < r['R6'] :
        zone = 6
    else:
        zone = 7
    return zone

In [41]:
_1_d_df['high_zone'] = _1_d_df.apply(lambda r: get_zone(r, 'High'), axis=1)
_1_d_df['low_zone'] = _1_d_df.apply(lambda r: get_zone(r, 'Low'), axis=1)

In [42]:
up_trend = _1_d_df[_1_d_df['long_trend'] == 'up']
down_trend = _1_d_df[_1_d_df['long_trend'] == 'down']

In [43]:
grph = _1_d_df.groupby('high_zone')[['High']]
irs = grph.count().reset_index().rename(columns={'High': 'sum'})
irs

,high_zone,sum
0,2,374
1,3,422
2,4,414
3,5,303


In [44]:
grpl = _1_d_df.groupby('low_zone')[['Low']]
irs = grpl.count().reset_index().rename(columns={'Low': 'sum'})
irs

,low_zone,sum
0,-5,374
1,-4,422
2,-3,414
3,-2,303
